In [1]:
from os import link
from bitarray import test
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
html_text = requests.get('https://www.imdb.com/chart/top/?ref_=nv_mv_250').text
soup = BeautifulSoup(html_text, 'lxml')
movies = soup.find_all('td', class_ = 'titleColumn')
names = []
links = []
for movie in movies:
    
    name = movie.find('a', href = True).text
    link =  movie.find('a', href = True)['href']

    names.append(name)
    links.append(link)

    print(f'''
    Movie Name: {name}
    Link: {"www.imdb.com" + link}
    ''')


    Movie Name: The Shawshank Redemption
    Link: www.imdb.com/title/tt0111161/
    

    Movie Name: The Godfather
    Link: www.imdb.com/title/tt0068646/
    

    Movie Name: The Dark Knight
    Link: www.imdb.com/title/tt0468569/
    

    Movie Name: The Godfather: Part II
    Link: www.imdb.com/title/tt0071562/
    

    Movie Name: 12 Angry Men
    Link: www.imdb.com/title/tt0050083/
    

    Movie Name: Schindler's List
    Link: www.imdb.com/title/tt0108052/
    

    Movie Name: The Lord of the Rings: The Return of the King
    Link: www.imdb.com/title/tt0167260/
    

    Movie Name: Pulp Fiction
    Link: www.imdb.com/title/tt0110912/
    

    Movie Name: The Lord of the Rings: The Fellowship of the Ring
    Link: www.imdb.com/title/tt0120737/
    

    Movie Name: The Good, the Bad and the Ugly
    Link: www.imdb.com/title/tt0060196/
    

    Movie Name: Forrest Gump
    Link: www.imdb.com/title/tt0109830/
    

    Movie Name: Fight Club
    Link: www.imdb.com/title/

In [3]:
zipped = list(zip(names, links))
movie_links = pd.DataFrame(zipped, columns=['Name', 'Link'])
print(movie_links)

                         Name                Link
0    The Shawshank Redemption   /title/tt0111161/
1               The Godfather   /title/tt0068646/
2             The Dark Knight   /title/tt0468569/
3      The Godfather: Part II   /title/tt0071562/
4                12 Angry Men   /title/tt0050083/
..                        ...                 ...
245                    Gandhi   /title/tt0083987/
246                   Aladdin   /title/tt0103639/
247                  Jai Bhim  /title/tt15097216/
248                  The Help   /title/tt1454029/
249      Beauty and the Beast   /title/tt0101414/

[250 rows x 2 columns]


In [23]:
age = ["all","sub18","18-29","30-44","44+"]
gender = ["_m", "_f"]
colnames = age
colnames.extend([i+gender[0] for i in age])
colnames.extend([i+gender[1] for i in age[:5]])
colnames.extend(["top_1000", "US", "non_US"])
print(colnames)
Ratings_df = pd.DataFrame(columns=colnames)
Voters_df = pd.DataFrame(columns=colnames)

['all', 'sub18', '18-29', '30-44', '44+', 'all_m', 'sub18_m', '18-29_m', '30-44_m', '44+_m', 'all_f', 'sub18_f', '18-29_f', '30-44_f', '44+_f', 'top_1000', 'US', 'non_US']


In [24]:
for name, link in zip(movie_links['Name'], movie_links['Link']):
    html = requests.get('https://www.imdb.com' + link + "ratings/").text
    soup2 = BeautifulSoup(html, 'lxml')


    # all ages -> "30-44" (excluding 45+)
    grid =  soup2.find_all('td', class_ = 'ratingTable') 
    ratings = []
    voters = []
    for cell in grid:

        rating = cell.find('div', class_ = 'bigcell').text.replace(' ','')
        voter = cell.find('a', href = True).text.replace(' ','').replace('\n', '')

        ratings.append(rating)
        voters.append(voter) 

    r_temp = pd.DataFrame([ratings], columns=colnames, index=[name])
    v_temp = pd.DataFrame([voters], columns=colnames, index=[name])

    Ratings_df = pd.concat([Ratings_df, r_temp])
    Voters_df =  pd.concat([Voters_df, v_temp])

In [25]:

display(Voters_df.head())
Ratings_df.head()

,all,sub18,18-29,30-44,44+,all_m,sub18_m,18-29_m,30-44_m,44+_m,all_f,sub18_f,18-29_f,30-44_f,44+_f,top_1000,US,non_US
The Shawshank Redemption,"2,616,796","1,298","368,505","1,008,855","266,620","1,466,571",960,"289,437","835,533","215,814","292,786",203,"66,237","157,436","44,810",894,"346,973","931,925"
The Godfather,"1,811,815",891,"234,297","718,024","207,413","1,065,306",653,"192,955","617,524","172,909","164,919",147,"33,319","89,473","29,906",885,"262,221","670,341"
The Dark Knight,"2,588,887","1,515","388,037","1,044,949","196,835","1,482,487","1,138","311,592","873,951","165,681","265,675",204,"63,044","155,202","26,951",928,"367,221","903,588"
The Godfather: Part II,"1,245,205",545,"154,666","501,966","154,057","754,846",386,"131,126","437,211","129,712","104,800",107,"18,930","57,235","21,168",848,"176,162","474,936"
12 Angry Men,"773,133",455,"115,278","295,948","86,249","449,860",347,"93,993","251,879","72,024","76,272",73,"18,105","40,293","12,612",785,"112,693","293,566"


,all,sub18,18-29,30-44,44+,all_m,sub18_m,18-29_m,30-44_m,44+_m,all_f,sub18_f,18-29_f,30-44_f,44+_f,top_1000,US,non_US
The Shawshank Redemption,9.3,9.3,9.3,9.3,9.1,9.3,9.4,9.4,9.3,9.1,9.2,9.0,9.3,9.2,9.2,8.9,9.3,9.2
The Godfather,9.2,9.2,9.2,9.2,9.2,9.2,9.3,9.3,9.2,9.2,8.9,9.0,8.9,8.9,9.0,9.1,9.2,9.2
The Dark Knight,9.0,9.2,9.2,9.0,8.4,9.0,9.2,9.3,9.1,8.4,8.7,8.6,8.9,8.7,8.2,8.5,9.1,8.9
The Godfather: Part II,9.0,9.1,9.1,9.0,9.0,9.0,9.2,9.1,9.0,9.0,8.8,8.6,8.8,8.7,8.8,8.8,9.1,9.0
12 Angry Men,9.0,9.3,9.1,9.0,8.8,9.0,9.4,9.2,9.0,8.8,8.9,8.9,9.0,8.9,8.8,8.7,8.9,9.0


In [ ]:
Voters_df.to_csv('Number_of_Voters.csv')
Ratings_df.to_csv('Ratings.csv')